In [ ]:
# Import Dependencies
import math
from flask import Flask, render_template, request, render_template_string, jsonify
import serial
from datetime import datetime
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from matplotlib.pyplot import style
style.use('fivethirtyeight')
%matplotlib inline

In [ ]:
# Mimic Inital BPM
initial_bpm = 80
start_pressed = False
i = 1
bpm = 0
data_analytics_file_name = ''

In [ ]:
# Open Serial COM Port
Arduino_serial = serial.Serial("COM4", 9600)

In [ ]:
app = Flask(__name__, template_folder='templates')


@app.route('/')
def index():
    return render_template('main.html', beats_per_minute= initial_bpm)


# Function to Save User Data to CSV File [WORKING]
def writeBPMData(fileName=None):
    # Convert Array of values to dataframe
    df = pd.DataFrame({'BPM':np.asarray(user_bpm_data), 'Time':user_bpm_time})
    df.to_csv(fileName, index=False)
    print('Saved user data as {}.csv.'.format(fileName))


# Function to get User Name and Gender [WORKING]
def name_and_gender(usrName=None):
    usr_name = usrName.split(',')[0]
    usr_gender = usrName.split(',')[1]
    print('User Name: {0}\t User Gender: {1}'.format(usr_name, usr_gender))
    with open('user_name.txt','w') as f:
        f.write(usrName)
        
        
# Function to Plot Heart Rate Data [WORKING]
def plotHeartBeat(file=None):
    plt.figure(figsize=(40,20))
    
    try:
        # Read Data from file and Show the Plot
        #hrt_data = numpy.loadtxt(file)
        hrt_data = np.genfromtxt(file, dtype=int) 
        print(hrt_data, type(hrt_data))
        name = file.split('.')[0] + '_heartRate.png'
        if (len(hrt_data) > 1):
            plt.plot(hrt_data, label='hrt_data', color="blue")
            plt.legend()
            plt.savefig(name)
            plt.show
    except:
        print('Need more data...')
        
        
# Function to Write data to CSV file and Plot in Real Time [WORKING]
def save_and_plot(fileName=None):
    threshold = 550
    x = 1
    usr_heart_rate = []
    usr_data_timestamp = []
    
    with open(fileName,'w') as f:
        while(x < 3000):
            x += 1
            ser_data = Arduino_serial.readline().strip()
            if (int(ser_data.decode()) > threshold):
                #print(ser_data.decode())
                usr_heart_rate.append(int(ser_data.decode()))
                usr_data_timestamp.append(str(datetime.time(datetime.now()))[:-7])
                f.write(ser_data.decode())
                f.write('\n')
    # Once all data is collected, plot it
    plotHeartBeat(file=fileName)
    
    name = fileName.split('.')[0]
    # Convert Array of values to dataframe
    df = pd.DataFrame({'Heart_Rate':np.asarray(usr_heart_rate), 'Time':np.asarray(usr_data_timestamp)})
    df.to_csv(name+'_analytics_data.csv', index=False)
    data_analytics_file_name = name+'_analytics_data.csv'
    print('Saved user data as {}.csv.'.format(name+'_analytics_data'))
                

# Function to get data Analytics
def get_data_analytics(userDataFile=None):
    print('\nGetting Data Analytics...\n')
    name = userDataFile.split('_')[0]
    
    df = pd.read_csv(userDataFile)
    #Calculate moving average with 0.75s in both directions, then append do dataset
    #One-sided window size, as proportion of the sampling frequency
    hrw = 0.75
    fs = 100
    
    # Calculate moving average
    mov_avg = df['Heart_Rate'].rolling(int(hrw*fs)).mean()
    # Impute where moving average function returns NaN, which is the beginning of the signal where x hrw
    avg_hr = (np.mean(df.Heart_Rate))
    mov_avg = [avg_hr if math.isnan(x) else x for x in mov_avg]
    # For now we raise the average by 20% to prevent the secondary heart contraction from interferce
    mov_avg = [x*1.2 for x in mov_avg]
    # Append the moving average to the dataframe
    df['Heart_Rate_Rollingmean'] = mov_avg
    
    #Mark regions of interest
    window = []
    peaklist = []
    listpos = 0 #We use a counter to move over the different data columns
    for datapoint in df.Heart_Rate:
        # Get local mean
        rollingmean = df.Heart_Rate_Rollingmean[listpos]
        # If no detectable R-complex activity -> do nothing
        if (datapoint < rollingmean) and (len(window) < 1):
            listpos += 1
            # If signal comes above local mean, mark ROI
        elif (datapoint > rollingmean):
            window.append(datapoint)
            listpos += 1
        else: # If signal drops below local mean -> determine highest point
            maximum = max(window)
            # Anotate the position of the point on the X-axis
            beatposition = listpos - len(window) + (window.index(max(window)))
            # Add detected peak to list
            peaklist.append(beatposition)
            # Clear marked ROI
            window = []
            listpos += 1
    
    ybeat = [df.Heart_Rate[x] for x in peaklist] #Get the y-value of all peaks for plotting purposes
    plt.figure(figsize=(40,20))
    plt.title("Detected Peaks in Signal and Moving Averages")
    plt.xlim(0,2500)
    plt.plot(df.Heart_Rate, alpha=0.5, color='blue', label='Heart_Rate') #Plot semi-transparent HR
    plt.plot(mov_avg, color ='green', label='Moving_avg') #Plot moving average
    plt.scatter(peaklist, ybeat, color='red', label='Peaks') #Plot detected peaks
    plt.legend(loc='best')
    plt.savefig('user_analysis.png')
    plt.show()
    
    print('\nCalculating Average Heart Rate...\n')
    RR_list = []
    cnt = 0
    while (cnt < (len(peaklist)-1)):
        RR_interval = (peaklist[cnt+1] - peaklist[cnt]) #Calculate distance between beats in # of samples
        ms_dist = ((RR_interval / fs) * 1000.0) #Convert sample distances to ms distances
        RR_list.append(ms_dist) #Append to list
        cnt += 1
    
    # 60000 ms (1 minute) / average R-R interval of signal
    bpm = 60000 / np.mean(RR_list)
    print("Average Heart Beat is: %.01f BPM" %bpm)
    plt.figure(figsize=(40,20))
    plt.title("Detected Peaks in Signal")
    plt.xlim(0,2500)
    plt.plot(df.Heart_Rate, alpha=0.5, color='blue', label="raw signal")
    plt.plot(mov_avg, color ='green', label="moving average")
    plt.scatter(peaklist, ybeat, color='red', label="average: %.1f BPM" %bpm)
    plt.legend(loc=4, framealpha=0.6)
    plt.savefig('user_avg_heartRate.png')
    plt.show()
    return bpm

            
        
@app.route('/', methods=['GET','POST'])
def data():
    print('Button Function...')
    # Variable to hold BPM Values
    bpm = 0
    text = ' '
    ser_start = False
    usr_name = ''
    
    # Format ( request.form[name] == value )
    if (request.method == 'POST'):
        
        # Get User Name and Gender for Generating Report
        if (request.form['button'] == 'user_name_submit'):
            user_name = request.form['user_name']
            name_and_gender(usrName=user_name)
        
        # Start Recording Data in CSV File and Plot Data in Real Time
        elif (request.form['button'] == 'start'):
            with open('user_name.txt','r') as f:
                text = f.read()
            print('NAME: ',text)
            name = text.split(',')[0]
            # Function to Start Reading the data
            save_and_plot(fileName=name+'.csv')
        
        elif (request.form['button'] == 'stop'):
            print('Preparing Data Analytics. . .')
            #Arduino_serial.close()
            with open('user_name.txt','r') as f:
                text = f.read()
            name = text.split(',')[0]
            file_name = name+'_analytics_data.csv'
            bpm = get_data_analytics(userDataFile=file_name)
            return render_template('main.html', beats_per_minute= round(bpm,2))
        
        elif (request.form['button'] == 'showPlot'):
            ser_data = Arduino_serial.readline().strip()
            
            if (ser_data > threshold):
                user_heart_beat_data.append(ser_data.decode())
    
        
    elif (request.method == 'GET'):
        return render_template('main.html',  beats_per_minute= bpm)
    
    print('End of Function...')
    #print(bpm, initial_bpm)
    return render_template('main.html', beats_per_minute= initial_bpm)


# Main Function
if __name__ == '__main__':
    app.run()